In [1]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [2]:
using SparseArrays, LinearAlgebra

In [3]:
struct ScalarField
    A::Vector{Matrix{Float64}}
    numElem::Vector{Int}
    nsteps::Int
    type::Symbol
    function ScalarField(A0, numElem0, nsteps0, type0)
        return new(A0, numElem0, nsteps0, type0)
    end
    function ScalarField(problem, dataField)
        if !isa(dataField, Vector)
            error("ScalarField: dataField are not arranged in a vector. Put them in [...]")
        end
        gmsh.model.setCurrent(problem.name)

        type = :scalarInElements
        nsteps = 1
        A = []
        numElem = Int[]
        pdim = 1
        for i in 1:length(dataField)
            phName, f, fx, fy, fz, fxy, fyz, fzx = dataField[i]
            dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
            for idm in 1:length(dimTags)
                dimTag = dimTags[idm]
                edim = dimTag[1]
                etag = dimTag[2]
                elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
                for i in 1:length(elemTypes)
                    et = elemTypes[i]
                    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                    sc1 = zeros(numNodes, nsteps)
                    for j in 1:length(elemTags[i])
                        elem = elemTags[i][j]
                        push!(numElem, elem)
                        for k in 1:numNodes
                            nodeTag = elemNodeTags[i][(j-1)*numNodes+k]
                            if f != :no
                                if isa(f, Function)
                                    coord, parametricCoord, dim, tag = gmsh.model.mesh.getNode(nodeTag)
                                    x = coord[1]
                                    y = coord[2]
                                    z = coord[3]
                                    ff = f(x, y, z)
                                else
                                    ff = f
                                end
                                sc1[k] = f
                            end
                        end
                        push!(A, sc1)
                    end
                end
            end
        end
        return new(A, numElem, nsteps, type)
    end
end

In [4]:
function nodePositionVector(problem)
    dim = problem.dim
    non = problem.non
    if dim != 3
        error("nodePositionVector: This function only works for 3D problems.")
    end
    r = zeros(non * dim)
    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            nodeTags, ncoord, parametricCoord = gmsh.model.mesh.getNodes(dim, -1, true, false)
            r[nodeTags*3 .- 2] = ncoord[1:3:length(ncoord)]
            r[nodeTags*3 .- 1] = ncoord[2:3:length(ncoord)]
            r[nodeTags*3 .- 0] = ncoord[3:3:length(ncoord)]
        end
    end
    return r
end

nodePositionVector (generic function with 1 method)

In [5]:
function stiffnessMatrixLinear(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            #=D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν ν 0 0 0;
                ν 1-ν ν 0 0 0;
                ν ν 1-ν 0 0 0;
                0 0 0 (1-2ν)/2 0 0;
                0 0 0 0 (1-2ν)/2 0;
                0 0 0 0 0 (1-2ν)/2]
                =#
            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            D = E / (1 - ν^2) * [1 ν 0;
                ν 1 0;
                0 0 (1-ν)/2]
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0;
                ν 1-ν 0;
                0 0 (1-2ν)/2]
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                #comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                #h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                #H = zeros(9 * numIntPoints, pdim * numNodes)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                K1 = zeros(pdim * numNodes, pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                sizehint!(∂H, 9 * numNodes)
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                            ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                            F = ∂H1 * r1
                            for l in 1:numNodes
                                B[k*rowsOfB-5, l*pdim-2] = F[1] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-2] = F[4] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-2] = F[7] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-2] = (F[4] * ∂h[1, (k-1)*numNodes+l] + F[1] * ∂h[2, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-1, l*pdim-2] = (F[7] * ∂h[2, (k-1)*numNodes+l] + F[4] * ∂h[3, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-0, l*pdim-2] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[7] * ∂h[1, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-5, l*pdim-1] = F[2] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-1] = F[5] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-1] = F[8] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-1] = (F[5] * ∂h[1, (k-1)*numNodes+l] + F[2] * ∂h[2, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-1, l*pdim-1] = (F[8] * ∂h[2, (k-1)*numNodes+l] + F[5] * ∂h[3, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-0, l*pdim-1] = (F[2] * ∂h[3, (k-1)*numNodes+l] + F[8] * ∂h[1, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-5, l*pdim-0] = F[3] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-0] = F[6] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-0] = F[9] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-0] = (F[6] * ∂h[1, (k-1)*numNodes+l] + F[3] * ∂h[2, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-1, l*pdim-0] = (F[9] * ∂h[2, (k-1)*numNodes+l] + F[6] * ∂h[3, (k-1)*numNodes+l]) / 2.0
                                B[k*rowsOfB-0, l*pdim-0] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[9] * ∂h[1, (k-1)*numNodes+l]) / 2.0
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    K1 .*= 0
                    for k in 1:numIntPoints
                        Ex = 10
                        νxy = 0.3
                        λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
                        μ = Ex / (2 * (1 + νxy))
                        #∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        #F1 = ∂H1 * r1
                        #F2 = reshape(F1, 3, 3)
                        #F2 = [1.0 0 0; 0 1.0 0; 0 0 1.0]
                        #J1 = det(F2)
                        #C2 = F2' * F2
                        #iC2 = inv(C2)
                        #iC1 = reshape(iC2, 9, 1)
                        #iCiC1 = iC1[[1, 5, 9, 4, 6, 3]] * iC1[[1, 5, 9, 4, 6, 3]]'
                        #i1 = [1, 2, 3, 1, 2, 3]
                        #j1 = [1, 2, 3, 2, 3, 1]
                        #k1 = [1, 2, 3, 1, 2, 3]
                        #l1 = [1, 2, 3, 2, 3, 1]
                        #II1 = zeros(6, 6)
                        #for i2 in 1:6, j2 in 1:6
                        #    II1[i2, j2] = (iC2[i1[i2], k1[j2]] * iC2[j1[i2], l1[j2]] + iC2[i1[i2], l1[j2]] * iC2[j1[i2], k1[j2]]) / 2
                        #end
                        #C1 = λ * iCiC1 + 2 * (μ - λ * log(J1)) * II1
                        λ0 = λ
                        μ0 = μ
                        C1 = [λ0+2μ0 λ0 λ0 0 0 0;
                            λ0 λ0+2μ0 λ0 0 0 0;
                            λ0 λ0 λ0+2μ0 0 0 0;
                            0 0 0 μ0 0 0;
                            0 0 0 0 μ0 0;
                            0 0 0 0 0 μ0]
                        B1 = B[k*rowsOfB-(rowsOfB-1):k*rowsOfB, 1:pdim*numNodes]
                        K1 += B1' * C1 * B1 * b * jacDet[k] * intWeights[k]
                    end
                    append!(I, nn2[Iidx[:]])
                    append!(J, nn2[Jidx[:]])
                    append!(V, K1[:])
                end
                push!(nn, nnet)
            end
        end
    end
    dof = problem.pdim * problem.non
    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return K
end

stiffnessMatrixLinear (generic function with 1 method)

In [6]:
function stiffnessMatrixNonLinear(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                H = spzeros(9 * numIntPoints, 9 * numNodes)
                sizehint!(H, 9 * numNodes * numIntPoints)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                K1 = zeros(pdim * numNodes, pdim * numNodes)
                nn2 = zeros(Int, dim * numNodes)
                nn9 = zeros(Int, 9 * numNodes)
                sizehint!(∂H, 9 * numNodes * numIntPoints)
                for k in 1:numIntPoints, l in 1:numNodes
                    for kk in 1:9
                        H[k*9-(9-kk), l*9-(9-kk)] = h[(k-1)*numNodes+l]
                    end
                end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:dim
                        nn2[k:dim:dim*numNodes] = dim * nnet[j, 1:numNodes] .- (dim - k)
                    end
                    for k in 1:9
                        nn9[k:9:9*numNodes] = 9 * nnet[j, 1:numNodes] .- (9 - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    #S1 = S[nn9]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    K1 .*= 0
                    for k in 1:numIntPoints
                        Ex = 10
                        νxy = 0.3
                        λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
                        μ = Ex / (2 * (1 + νxy))
                        I3 = [1 0 0; 0 1 0; 0 0 1]
                        #H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        ∂H0 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        F1 = ∂H0 * r1
                        F2 = reshape(F1, 3, 3)
                        #J1 = det(F2)
                        E = (F2' * F2 - I3) / 2
                        #iC = inv(C)
                        S2 = 2μ * E + λ * (E[1, 1] + E[2, 2] + E[3, 3]) * I3
                        #S0 = reshape(H1 * S1, 3, 3)
                        ∂H1 = ∂H[k*9-(9-1):3:k*9-(9-9), 1:pdim*numNodes]
                        ∂H2 = ∂H[k*9-(9-2):3:k*9-(9-9), 1:pdim*numNodes]
                        ∂H3 = ∂H[k*9-(9-3):3:k*9-(9-9), 1:pdim*numNodes]
                        K1 += (∂H1' * S2 * ∂H1 + ∂H2' * S2 * ∂H2 + ∂H3' * S2 * ∂H3) * b * jacDet[k] * intWeights[k]
                    end
                    append!(I, nn2[Iidx[:]])
                    append!(J, nn2[Jidx[:]])
                    append!(V, K1[:])
                end
                push!(nn, nnet)
            end
        end
    end
    dof = problem.pdim * problem.non
    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return K
end

stiffnessMatrixNonLinear (generic function with 1 method)

In [7]:
function loadVectorNonLinear(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    f = zeros(problem.non * problem.pdim)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν ν 0 0 0;
                ν 1-ν ν 0 0 0;
                ν ν 1-ν 0 0 0;
                0 0 0 (1-2ν)/2 0 0;
                0 0 0 0 (1-2ν)/2 0;
                0 0 0 0 0 (1-2ν)/2]

            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            D = E / (1 - ν^2) * [1 ν 0;
                ν 1 0;
                0 0 (1-ν)/2]
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0;
                ν 1-ν 0;
                0 0 (1-2ν)/2]
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                H = spzeros(9 * numIntPoints, 9 * numNodes)
                sizehint!(H, 9 * numNodes * numIntPoints)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                f1 = zeros(pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                nn9 = zeros(Int, 9 * numNodes)
                sizehint!(∂H, 9 * numNodes * numIntPoints)
                #for k in 1:numIntPoints, l in 1:numNodes
                #    for kk in 1:3:9, ll in 1:3
                #H[k*9-(9-kk)-1+ll, l*pdim-(pdim-ll)] = h[(k-1)*numNodes+l]
                #    end
                #end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    for k in 1:9
                        nn9[k:9:9*numNodes] = 9 * nnet[j, 1:numNodes] .- (9 - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    #S1 = S[nn9]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                            ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                            F = ∂H1 * r1
                            for l in 1:numNodes
                                B[k*rowsOfB-5, l*pdim-2] = F[1] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-2] = F[4] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-2] = F[7] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-2] = (F[4] * ∂h[1, (k-1)*numNodes+l] + F[1] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-2] = (F[7] * ∂h[2, (k-1)*numNodes+l] + F[4] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-2] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[7] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-1] = F[2] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-1] = F[5] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-1] = F[8] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-1] = (F[5] * ∂h[1, (k-1)*numNodes+l] + F[2] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-1] = (F[8] * ∂h[2, (k-1)*numNodes+l] + F[5] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-1] = (F[2] * ∂h[3, (k-1)*numNodes+l] + F[8] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-0] = F[3] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-0] = F[6] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-0] = F[9] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-0] = (F[6] * ∂h[1, (k-1)*numNodes+l] + F[3] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-0] = (F[9] * ∂h[2, (k-1)*numNodes+l] + F[6] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-0] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[9] * ∂h[1, (k-1)*numNodes+l]) / 2
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    f1 .*= 0
                    for k in 1:numIntPoints
                        Ex = 10
                        νxy = 0.3
                        λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
                        μ = Ex / (2 * (1 + νxy))
                        I3 = [1 0 0; 0 1 0; 0 0 1]
                        #H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        ∂H0 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        F1 = ∂H0 * r1
                        F2 = reshape(F1, 3, 3)
                        #J1 = det(F2)
                        E = (F2' * F2 - I3) / 2
                        #iC = inv(C)
                        S2 = 2μ * E + λ * (E[1, 1] + E[2, 2] + E[3, 3]) * I3
                        #S2 = [1 0 0; 0 0 0; 0 0 0]
                        #S2 = -I3
                        #S3 = reshape(S2, 9, 1)
                        #H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        #S0 = H1 * S1
                        B1 = B[k*rowsOfB-(rowsOfB-1):k*rowsOfB, 1:pdim*numNodes]
                        f1 += B1' * S2[[1, 5, 9, 4, 6, 3]] * b * jacDet[k] * intWeights[k]
                    end
                    f[nn2] += f1
                end
            end
        end
    end
    return f
end

loadVectorNonLinear (generic function with 1 method)

In [8]:
function followerLoadVector(problem, r, loads)
    gmsh.model.setCurrent(problem.name)
    if !isa(loads, Vector)
        error("loadVector: loads are not arranged in a vector. Put them in [...]")
    end
    pdim = problem.pdim
    DIM = problem.dim
    b = problem.thickness
    non = problem.non
    dof = pdim * non
    fp = zeros(dof)
    ncoord2 = zeros(3 * problem.non)
    F0 = deformationGradient(problem, r)
    F0 = FEM.elementsToNodes(problem, F0)
    for n in 1:length(loads)
        name, fx, fy, fz = loads[n]
        if pdim == 3
            f = [.0, .0, .0]
        elseif pdim == 2
            f = [.0, .0]
        elseif pdim == 1
            f = [.0]
        else
            error("loadVector: dimension of the problem is $(problem.dim).")
        end
        dimTags = gmsh.model.getEntitiesForPhysicalName(name)
        for i ∈ 1:length(dimTags)
            dimTag = dimTags[i]
            dim = dimTag[1]
            tag = dimTag[2]
            elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(dim, tag)
            nodeTags::Vector{Int64}, ncoord, parametricCoord = gmsh.model.mesh.getNodes(dim, tag, true, false)
            ncoord2[nodeTags*3 .- 2] = ncoord[1:3:length(ncoord)]
            ncoord2[nodeTags*3 .- 1] = ncoord[2:3:length(ncoord)]
            ncoord2[nodeTags*3 .- 0] = ncoord[3:3:length(ncoord)]
            for ii in 1:length(elementTypes)
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[ii])
                nnoe = reshape(elemNodeTags[ii], numNodes, :)'
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[ii], "Gauss" * string(order+1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[ii], intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[ii], intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elementTags[ii]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                H = zeros(pdim * numIntPoints, pdim * numNodes)
                HH = zeros(9 * numIntPoints, 9 * numNodes)
                for j in 1:numIntPoints
                    for k in 1:numNodes
                        for l in 1:pdim
                            H[j*pdim-(pdim-l), k*pdim-(pdim-l)] = h[k, j]
                        end
                        for l in 1:9
                            HH[j*9-(9-l), k*9-(9-l)] = h[(j-1)*numNodes+k]
                        end
                    end
                end
                f1 = zeros(pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                nn9 = zeros(Int, 9 * numNodes)
                ∂h = zeros(pdim, numNodes * numIntPoints)
                for l in 1:length(elementTags[ii])
                    elem = elementTags[ii][l]
                    for k in 1:numNodes
                        nnet[l, k] = elemNodeTags[ii][(l-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnoe[l, 1:numNodes] .- (pdim - k)
                    end
                    for k in 1:9
                        nn9[k:9:9*numNodes] = 9 * nnet[l, 1:numNodes] .- (9 - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    f1 .*= 0
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    F9 = F0[nn9]
                    for j in 1:numIntPoints
                        H9 = HH[j*9-(9-1):j*9, 1:9*numNodes]
                        F1 = H9 * F9
                        F1 = reshape(F1, 3, 3)
                        x = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 2]
                        y = 0
                        z = 0
                        if isa(fx, Function) || isa(fy, Function) || isa(fz, Function)
                            y = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 1]
                            z = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 0]
                        end
                        if fz == 2im
                            if isa(fx, Function)
                                error("heatConvectionVector: h cannot be a function.")
                            end
                            f[1] = isa(fy, Function) ? fx * fy(x, y, z) : fx * fy
                        else
                            f[1] = isa(fx, Function) ? fx(x, y, z) : fx
                        end
                        if pdim > 1
                            f[2] = isa(fy, Function) ? fy(x, y, z) : fy
                        end
                        if pdim == 3
                            f[3] = isa(fz, Function) ? fz(x, y, z) : fz
                        end
                        H1 = H[j*pdim-(pdim-1):j*pdim, 1:pdim*numNodes] # H1[...] .= H[...] ????
                        ############### NANSON ######## 3D ###################################
                        if DIM == 3 && dim == 3
                            Ja = jacDet[j]
                        elseif DIM == 3 && dim == 2
                            xy = Jac[1, 3*j-2] * Jac[2, 3*j-1] - Jac[2, 3*j-2] * Jac[1, 3*j-1]
                            yz = Jac[2, 3*j-2] * Jac[3, 3*j-1] - Jac[3, 3*j-2] * Jac[2, 3*j-1]
                            zx = Jac[3, 3*j-2] * Jac[1, 3*j-1] - Jac[1, 3*j-2] * Jac[3, 3*j-1]
                            Ja = √(xy^2 + yz^2 + zx^2)
                        elseif DIM == 3 && dim == 1
                            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2 + (Jac[3, 3*j-2])^2)
                        elseif DIM == 3 && dim == 0
                            Ja = 1
                        ############ 2D #######################################################
                        elseif DIM == 2 && dim == 2 && problem.type != :AxiSymmetric && problem.type != :AxiSymmetricHeatConduction
                            Ja = jacDet[j] * b
                        elseif DIM == 2 && dim == 2 && (problem.type == :AxiSymmetric || problem.type == :AxiSymmetricHeatConduction)
                            Ja = 2π * jacDet[j] * x
                        elseif DIM == 2 && dim == 1 && problem.type != :AxiSymmetric && problem.type != :AxiSymmetricHeatConduction
                            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2) * b
                        elseif DIM == 2 && dim == 1 && (problem.type == :AxiSymmetric || problem.type == :AxiSymmetricHeatConduction)
                            Ja = 2π * √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2) * x
                        elseif DIM == 2 && dim == 0
                            Ja = 1
                        ############ 1D #######################################################
                        else
                            error("loadVector: dimension of the problem is $(problem.dim), dimension of load is $dim.")
                        end
                        f1 += H1' * F1 * f * Ja * intWeights[j]
                    end
                    fp[nn2] += f1
                end
            end
        end
    end
    return fp
end



followerLoadVector (generic function with 1 method)

In [9]:
function deformationGradient(problem, r)
    gmsh.model.setCurrent(problem.name)

    type = :F
    nsteps = size(r, 2)
    F = []
    numElem = Int[]
    ncoord2 = zeros(3 * problem.non)
    dim = problem.dim
    pdim = problem.pdim
    non = problem.non

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim != 3 || problem.type != :Solid
            error("deformationGradient: dimension is $(problem.dim), problem type is $(problem.type). (They must be 3 and :Solid.)")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                F1 = zeros(9numNodes, nsteps)
                nodeCoord = zeros(numNodes * 3)
                for k in 1:dim, j = 1:numNodes
                    nodeCoord[k+(j-1)*3] = localNodeCoord[k+(j-1)*dim]
                end
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, nodeCoord, "GradLagrange")
                ∇h = reshape(dfun, :, numNodes)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numNodes)
                ∂h = zeros(dim, numNodes * numNodes)
                ∂H = spzeros(9 * numNodes, pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                sizehint!(∂H, 9 * numNodes)
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, nodeCoord)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numNodes
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numNodes, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    ∂H .*= 0
                    for k in 1:numNodes
                        for l in 1:numNodes
                            ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                            ∂H[k*9-(9-2), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                            ∂H[k*9-(9-3), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                            ∂H[k*9-(9-4), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                            ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                            ∂H[k*9-(9-6), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                            ∂H[k*9-(9-7), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                            ∂H[k*9-(9-8), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                            ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                        end
                    end
                    push!(numElem, elem)
                    for k in 1:numNodes
                        ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        for kk in 1:nsteps
                            F0 = ∂H1 * r[nn2, kk]
                            F1[(k-1)*9+1:k*9, kk] = [F0[1], F0[4], F0[7],
                                F0[2], F0[5], F0[8],
                                F0[3], F0[6], F0[9]]
                        end
                    end
                    push!(F, F1)
                end
            end
        end
    end
    sigma = FEM.TensorField(F, numElem, nsteps, type)
    return sigma
end

deformationGradient (generic function with 1 method)

In [10]:
import Base.*
function *(A::FEM.TensorField, B::FEM.TensorField)
    if (A.type == :s || A.type == :e || A.type == :F) && (B.type == :s || B.type == :e || B.type == :F)
        if length(A.A) != length(B.A)
            error("*(A::TensoeField, B::TensorField): size of A=$(length(A.A)) != size of B=$(length(B.A))")
        end
        if A.numElem != B.numElem
            error("*(A::TensoeField, B::TensorField): tensor fields are not compatible.")
        end
        nsteps = A.nsteps
        nsteps2 = B.nsteps
        if nsteps != nsteps2
            error("*(A::TensoeField, B::TensorField): nsteps of A=$(A.nsteps) != nsteps of B=$(B.nsteps)")
        end
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            m = length(B.A[i]) ÷ 9
            if n != m
                error("*(A::TensoeField, B::TensorField): size of A.A[$i]=$(9n) != size of B.A[$j]=$(9m)")
            end
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(reshape(A.A[i][9j-8:9j, k], 3, 3) * reshape(B.A[i][9j-8:9j, k], 3, 3), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("*(A::TensorField, B::TensorField): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

function *(A::ScalarField, B::FEM.TensorField)
    sz = 0
    nsteps = B.nsteps
    sec = intersect(B.numElem, A.numElem)
    indS = []
    indT = []
    sizehint!(indS, length(sec))
    sizehint!(indT, length(sec))
    for i in sec
        append!(indS, findall(j -> j == i, A.numElem))
        append!(indT, findall(j -> j == i, B.numElem))
    end
    C = []
    num = []
    sizehint!(C, length(sec))
    sizehint!(num, length(sec))
    D = []
    for i in eachindex(sec)
        n = length(B.A[i]) ÷ 9
        if n != sz
            D = zeros(9n, nsteps)
            sz = n
        end
        for j in 1:n
            for k in 1:nsteps
                D[9j-8:9j, k] = A.A[indS[i]][j, k] * B.A[indT[i]][9j-8:9j, k]
            end
        end
        append!(num, sec[i])
        push!(C, D)
    end
    return FEM.TensorField(C, num, B.nsteps, :e)
end

function *(B::FEM.TensorField, A::ScalarField)
    sz = 0
    nsteps = B.nsteps
    sec = intersect(B.numElem, A.numElem)
    indS = []
    indT = []
    sizehint!(indS, length(sec))
    sizehint!(indT, length(sec))
    for i in sec
        append!(indS, findall(j -> j == i, A.numElem))
        append!(indT, findall(j -> j == i, B.numElem))
    end
    C = []
    num = []
    sizehint!(C, length(sec))
    sizehint!(num, length(sec))
    D = []
    for i in eachindex(sec)
        n = length(B.A[i]) ÷ 9
        if n != sz
            D = zeros(9n, nsteps)
            sz = n
        end
        for j in 1:n
            for k in 1:nsteps
                D[9j-8:9j, k] = A.A[indS[i]][j, k] * B.A[indT[i]][9j-8:9j, k]
            end
        end
        append!(num, sec[i])
        push!(C, D)
    end
    return FEM.TensorField(C, num, B.nsteps, :e)
end

import Base./
function /(B::FEM.TensorField, A::ScalarField)
    sz = 0
    nsteps = B.nsteps
    sec = intersect(B.numElem, A.numElem)
    indS = []
    indT = []
    sizehint!(indS, length(sec))
    sizehint!(indT, length(sec))
    for i in sec
        append!(indS, findall(j -> j == i, A.numElem))
        append!(indT, findall(j -> j == i, B.numElem))
    end
    C = []
    num = []
    sizehint!(C, length(sec))
    sizehint!(num, length(sec))
    D = []
    for i in eachindex(sec)
        n = length(B.A[i]) ÷ 9
        if n != sz
            D = zeros(9n, nsteps)
            sz = n
        end
        for j in 1:n
            for k in 1:nsteps
                D[9j-8:9j, k] = B.A[indT[i]][9j-8:9j, k] / A.A[indS[i]][j, k]
            end
        end
        append!(num, sec[i])
        push!(C, D)
    end
    return FEM.TensorField(C, num, B.nsteps, :e)
end

function *(A::ScalarField, B::FEM.ScalarField)
    sz = 0
    nsteps = B.nsteps
    sec = intersect(B.numElem, A.numElem)
    indS = []
    indT = []
    sizehint!(indS, length(sec))
    sizehint!(indT, length(sec))
    for i in sec
        append!(indS, findall(j -> j == i, A.numElem))
        append!(indT, findall(j -> j == i, B.numElem))
    end
    C = []
    num = []
    sizehint!(C, length(sec))
    sizehint!(num, length(sec))
    D = []
    for i in eachindex(sec)
        n = length(B.A[i])
        if n != sz
            D = zeros(n, nsteps)
            sz = n
        end
        for j in 1:n
            for k in 1:nsteps
                D[j, k] = A.A[indS[i]][j, k] * B.A[indT[i]][j, k]
            end
        end
        append!(num, sec[i])
        push!(C, D)
    end
    return FEM.ScalarField(C, num, B.nsteps, :e)
end

function /(A::ScalarField, B::ScalarField)
    sz = 0
    nsteps = B.nsteps
    sec = intersect(B.numElem, A.numElem)
    indS = []
    indT = []
    sizehint!(indS, length(sec))
    sizehint!(indT, length(sec))
    for i in sec
        append!(indS, findall(j -> j == i, A.numElem))
        append!(indT, findall(j -> j == i, B.numElem))
    end
    C = []
    num = []
    sizehint!(C, length(sec))
    sizehint!(num, length(sec))
    D = []
    for i in eachindex(sec)
        n = length(B.A[i])
        if n != sz
            D = zeros(n, nsteps)
            sz = n
        end
        for j in 1:n
            for k in 1:nsteps
                D[j, k] = A.A[indS[i]][j, k] / B.A[indT[i]][j, k]
            end
        end
        append!(num, sec[i])
        push!(C, D)
    end
    return ScalarField(C, num, B.nsteps, :e)
end

import Base.transpose
function transpose(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(transpose(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("transpose(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

import Base.adjoint
function adjoint(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(adjoint(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("adjoint(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

function unitTensor(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape([1 0 0; 0 1 0; 0 0 1], 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("unit(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

function traceI(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    trace = A.A[i][9j-8, k] + A.A[i][9j-4, k] + A.A[i][9j, k]
                    D[9j-8:9j, k] = reshape([trace 0 0; 0 trace 0; 0 0 trace], 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("traceI(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

function trace(A::FEM.TensorField)
    sz = 0
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        D = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            if sz != n
                D = zeros(n, nsteps)
                sz = n
            end
            for j in 1:n
                for k in 1:nsteps
                    trace = A.A[i][9j-8, k] + A.A[i][9j-4, k] + A.A[i][9j, k]
                    D[j, k] = trace
                end
            end
            push!(C, D)
        end
        return FEM.ScalarField(C, A.numElem, A.nsteps, :e)
    else
        error("trace(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

function detI(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    d = det(reshape(A.A[i][9j-8:9j, k], 3, 3))
                    D[9j-8:9j, k] = reshape([d 0 0; 0 d 0; 0 0 d], 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("detI(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

import LinearAlgebra.det
function det(A::FEM.TensorField)
    sz = 0
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        D = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            if sz != n
                D = zeros(n, nsteps)
                sz = n
            end
            for j in 1:n
                for k in 1:nsteps
                    d = LinearAlgebra.det(reshape(A.A[i][9j-8:9j, k], 3, 3))
                    D[j, k] = d
                end
            end
            push!(C, D)
        end
        return ScalarField(C, A.numElem, A.nsteps, :sc)
    else
        error("det(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

import Base.+
function +(A::FEM.TensorField, B::FEM.TensorField)
    if (A.type == :s || A.type == :e || A.type == :F) && (B.type == :s || B.type == :e || B.type == :F)
        if length(A.A) != length(B.A)
            error("+(A::TensoeField, B::TensorField): size of A=$(length(A.A)) != size of B=$(length(B.A))")
        end
        nsteps = A.nsteps
        nsteps2 = B.nsteps
        if nsteps != nsteps2
            error("+(A::TensoeField, B::TensorField): nsteps of A=$(A.nsteps) != nsteps of B=$(B.nsteps)")
        end
        sec = intersect(A.numElem, B.numElem)
        ind1 = []
        ind2 = []
        sizehint!(ind1, length(sec))
        sizehint!(ind2, length(sec))
        for i in sec
            append!(ind1, findall(j -> j == i, A.numElem))
            append!(ind2, findall(j -> j == i, B.numElem))
        end
        dif1 = setdiff(A.numElem, B.numElem)
        ind3 = []
        sizehint!(ind3, length(dif1))
        for i in dif1
            append!(ind3, findall(j -> j == i, A.numElem))
        end
        dif2 = setdiff(B.numElem, A.numElem)
        ind4 = []
        sizehint!(ind4, length(dif2))
        for i in dif2
            append!(ind4, findall(j -> j == i, B.numElem))
        end
        C = []
        num = []
        sizehint!(C, length(sec) + length(dif1) + length(dif2))
        sizehint!(num, length(sec) + length(dif1) + length(dif2))
        for i in eachindex(sec)
            #n = length(A.A[i]) ÷ 9
            #m = length(B.A[i]) ÷ 9
            #if n != m
            #    error("+(A::TensoeField, B::TensorField): size of A.A[$i]=$(9n) != size of B.A[$j]=$(9m)")
            #end
            D = A.A[i] + B.A[i]
            append!(num, sec[i])
            push!(C, D)
        end
        for i in eachindex(dif1)
            D = A.A[i]
            append!(num, dif1[i])
            push!(C, D)
        end
        for i in eachindex(dif2)
            D = B.A[i]
            append!(num, dif2[i])
            push!(C, D)
        end
        return FEM.TensorField(C, num, A.nsteps, :e)
    else
        error("+(A::TensorField, B::TensorField): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

import Base.-
function -(A::FEM.TensorField, B::FEM.TensorField)
    if (A.type == :s || A.type == :e || A.type == :F) && (B.type == :s || B.type == :e || B.type == :F)
        if length(A.A) != length(B.A)
            error("-(A::TensoeField, B::TensorField): size of A=$(length(A.A)) != size of B=$(length(B.A))")
        end
        nsteps = A.nsteps
        nsteps2 = B.nsteps
        if nsteps != nsteps2
            error("-(A::TensoeField, B::TensorField): nsteps of A=$(A.nsteps) != nsteps of B=$(B.nsteps)")
        end
        sec = intersect(A.numElem, B.numElem)
        ind1 = []
        ind2 = []
        sizehint!(ind1, length(sec))
        sizehint!(ind2, length(sec))
        for i in sec
            append!(ind1, findall(j -> j == i, A.numElem))
            append!(ind2, findall(j -> j == i, B.numElem))
        end
        dif1 = setdiff(A.numElem, B.numElem)
        ind3 = []
        sizehint!(ind3, length(dif1))
        for i in dif1
            append!(ind3, findall(j -> j == i, A.numElem))
        end
        dif2 = setdiff(B.numElem, A.numElem)
        ind4 = []
        sizehint!(ind4, length(dif2))
        for i in dif2
            append!(ind4, findall(j -> j == i, B.numElem))
        end
        C = []
        num = []
        sizehint!(C, length(sec) + length(dif1) + length(dif2))
        sizehint!(num, length(sec) + length(dif1) + length(dif2))
        for i in eachindex(sec)
            D = A.A[i] - B.A[i]
            append!(num, sec[i])
            push!(C, D)
        end
        for i in eachindex(dif1)
            D = A.A[i]
            append!(num, dif1[i])
            push!(C, D)
        end
        for i in eachindex(dif2)
            D = -B.A[i]
            append!(num, dif2[i])
            push!(C, D)
        end
        return FEM.TensorField(C, num, A.nsteps, :e)
    else
        error("-(A::TensorField, B::TensorField): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

#=
import Base.-
function -(A::FEM.TensorField, B::FEM.TensorField)
    if (A.type == :s || A.type == :e || A.type == :F) && (B.type == :s || B.type == :e || B.type == :F)
        if length(A.A) != length(B.A)
            error("-(A::TensoeField, B::TensorField): size of A=$(length(A.A)) != size of B=$(length(B.A))")
        end
        nsteps = A.nsteps
        nsteps2 = B.nsteps
        if nsteps != nsteps2
            error("*(A::TensoeField, B::TensorField): nsteps of A=$(A.nsteps) != nsteps of B=$(B.nsteps)")
        end
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            m = length(B.A[i]) ÷ 9
            if n != m
                error("-(A::TensoeField, B::TensorField): size of A.A[$i]=$(9n) != size of B.A[$j]=$(9m)")
            end
            D = A.A[i] - B.A[i]
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("-(A::TensorField, B::TensorField): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end
=#

function *(A::FEM.TensorField, b)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            D = A.A[i] * b
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("*(A::TensorField, b): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

function *(b, A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            D = A.A[i] * b
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("*(A::TensorField, b): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

function *(A::ScalarField, b)
    nsteps = A.nsteps
    C = []
    for i in 1:length(A.A)
        D = A.A[i] * b
        push!(C, D)
    end
    return FEM.ScalarField(C, A.numElem, A.nsteps, :e)
end

function *(b, A::ScalarField)
    C = []
    for i in 1:length(A.A)
        D = A.A[i] * b
        push!(C, D)
    end
    return ScalarField(C, A.numElem, A.nsteps, :e)
end

function /(A::FEM.TensorField, b)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            D = A.A[i] / b
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("/(A::TensorField, b): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

function /(A::FEM.ScalarField, b)
    nsteps = A.nsteps
    C = []
    for i in 1:length(A.A)
        D = A.A[i] / b
        push!(C, D)
    end
    return FEM.ScalarField(C, A.numElem, A.nsteps, :e)
end

function /(b, A::FEM.ScalarField)
    nsteps = A.nsteps
    C = []
    for i in 1:length(A.A)
        D = A.A[i] ./ b
        push!(C, D)
    end
    return FEM.ScalarField(C, A.numElem, A.nsteps, :e)
end

import Base.inv
function inv(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(inv(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("inv(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

import Base.sqrt
function sqrt(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(sqrt(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("sqrt(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

import Base.log
function log(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(log(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("log(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

import Base.log
function log(A::ScalarField)
    nsteps = A.nsteps
    C = []
    for i in 1:length(A.A)
        n = length(A.A[i])
        D = zeros(n, nsteps)
        for j in 1:n
            for k in 1:nsteps
                D[j, k] = log(A.A[i][j, k])
            end
        end
        push!(C, D)
    end
    return ScalarField(C, A.numElem, A.nsteps, :e)
end


log (generic function with 28 methods)

In [11]:
gmsh.open("body1.geo")

Info    : Reading 'body1.geo'...
Info    : Meshing 1D...                                                                                                                       
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000646875s, CPU 0.000648s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [

In [12]:
λ = 1
μ = 2

2

In [13]:
#mat = FEM.material("body", E=μ * (3λ + 2μ) / (λ + μ), ν=λ / 2 / (λ + μ))
mat = FEM.material("body", E=10, ν=0.3)
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 7)


LowLevelFEM.Problem("body1", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 10.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 5.769230769230768, 3.846153846153846, 8.333333333333334)], 1.0, 8)

In [14]:
field = FEM.field("body", f=2)
sc = ScalarField(problem, [field])

ScalarField([[2.0; 2.0; … ; 2.0; 2.0;;]], [27], 1, :scalarInElements)

In [15]:
r0 = nodePositionVector(problem)
F = deformationGradient(problem, r0)

LowLevelFEM.TensorField([[1.0; 0.0; … ; 0.0; 1.0;;]], [27], 1, :F)

In [16]:
sc * F

LowLevelFEM.TensorField([[2.0; 0.0; … ; 0.0; 2.0;;]], [27], 1, :e)

In [17]:
supp = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
traction = FEM.load("right", fy=-0.001)
bodyforce = FEM.load("body", fx=1)

q = FEM.solveDisplacement(problem, [traction], [supp])
S = FEM.solveStress(problem, q)

LowLevelFEM.TensorField([[0.002533333333333336; -0.0017238095238095248; … ; 0.00034285714285714274; -0.00011428571428571363;;]], [27], 1, :s)

In [18]:
u1 = FEM.showDoFResults(problem, q, :uvec)
S1 = FEM.showElementResults(problem, S, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [19]:
f = FEM.loadVector(problem, [traction])
r0 = nodePositionVector(problem)
n = 201
r = zeros(length(r0), n + 1)
q0 = zeros(length(r0), n)
r[:, 1] = r0
err = 1
ε = 1e-3
nlimit = 200

i = 0
for j in range(1, 10)
    display("load step: $j")
    err = 1
    while err > ε && i < nlimit
        i += 1
        #F = deformationGradient(problem, r[:, i])

        Kl = stiffnessMatrixLinear(problem, r[:, i])
        Knl = stiffnessMatrixNonLinear(problem, r[:, i])
        #f = followerLoadVector(problem, r[:, i], [traction])
        fnl = loadVectorNonLinear(problem, r[:, i])
        K1, f1 = FEM.applyBoundaryConditions(problem, Kl + Knl, f * j / 10 - fnl, [supp])
        q = FEM.solveDisplacement(K1, f1)
        r[:, i+1] = r[:, i] + q
        err = maximum(abs.(q)) / abs(maximum(r0) - minimum(r0))
        display(err)
    end
end

for i in 1:n+1
    r[:, i] -= r0
end
u2 = FEM.showDoFResults(problem, r, :uvec, t=1:n+1, visible=true)

"load step: 1"

0.00013062507741674764

"load step: 2"

2.5030321643815155e-5

"load step: 3"

...

"load step: 9"

0.00012909940004641522

"load step: 10"

2.5090904204263613e-5

3

In [20]:
r = nodePositionVector(problem)
ux(x, y, z) = 0.4x
uy(x, y, z) = 0.5y
uz(x, y, z) = 0.6z
q0 = FEM.field("body", fx=ux, fy=uy, fz=uz)
q = FEM.vectorField(problem, [q0])
F = deformationGradient(problem, q)

LowLevelFEM.TensorField([[0.4; 0.0; … ; 0.0; 0.6;;]], [27], 1, :F)

In [21]:
reshape(F.A[1][1:9], 3, 3)

3×3 Matrix{Float64}:
 0.4  0.0  0.0
 0.0  0.5  0.0
 0.0  0.0  0.6

In [22]:
E = (F' * F - unitTensor(F)) / 2

reshape(E.A[1][1:9], 3, 3)

3×3 Matrix{Float64}:
 -0.42   0.0     0.0
  0.0   -0.375   0.0
  0.0    0.0    -0.32

In [23]:
e = (unitTensor(F) - inv(F * F')) / 2

reshape(e.A[1][1:9], 3, 3)

3×3 Matrix{Float64}:
 -2.625   0.0   0.0
  0.0    -1.5   0.0
  0.0     0.0  -0.888889

In [24]:
e = inv(F') * E * inv(F)

reshape(e.A[1][1:9], 3, 3)

3×3 Matrix{Float64}:
 -2.625   0.0   0.0
  0.0    -1.5   0.0
  0.0     0.0  -0.888889

In [25]:
U = sqrt(F' * F)

reshape(U.A[1][1:9], 3, 3)

3×3 Matrix{Float64}:
 0.4  0.0  0.0
 0.0  0.5  0.0
 0.0  0.0  0.6

In [26]:
Ex = 10
νxy = 0.3
λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
μ = Ex / (2 * (1 + νxy))
I3 = unitTensor(F)
iC = inv(F' * F)
J1 = det(F)
SII = μ * (I3 - iC) + λ * log(J1) / J1 * I3

reshape(SII.A[1][1:9], 3, 3)

3×3 Matrix{Float64}:
 -122.128     0.0       0.0
    0.0    -113.474     0.0
    0.0       0.0    -108.773

In [27]:
function probe(A, x, y, z)
    elementTag, elementType, nodeTags, u, v, w = gmsh.model.mesh.getElementByCoordinates(x, y, z, 3, false)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementType)
    comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementType, [u, v, w], "Lagrange")
    SS = [0.0, 0, 0, 0, 0, 0, 0, 0, 0]
    if isa(A, FEM.TensorField)
        ind = findfirst(i -> i == elementTag, SII.numElem)
        for i in range(1, 9)
            SS[i] = fun' * SII.A[ind][i:9:9numNodes, 1]
        end
    elseif isa(A, Matrix)
        for i in range(1, 9)
            SS[i] = fun' * A[9nodeTags.-(9-i), 1]
        end
    end
    return reshape(SS, 3, 3)
end

probe (generic function with 1 method)

In [28]:
SII2 = FEM.elementsToNodes(problem, SII)

72×1 Matrix{Float64}:
 -122.12805462500435
    0.0
    0.0
    0.0
 -113.4742084711582
    0.0
    0.0
    0.0
 -108.77335377030352
 -122.12805462500435
    ⋮
 -122.12805462500435
    0.0
    0.0
    0.0
 -113.4742084711582
    0.0
    0.0
    0.0
 -108.77335377030352

In [29]:
probe(SII2, 0, 0, 0)

3×3 Matrix{Float64}:
 -122.128     0.0       0.0
    0.0    -113.474     0.0
    0.0       0.0    -108.773

In [30]:
field = FEM.field("right", f=2)
sc = ScalarField(problem, [field])

ScalarField([[2.0; 2.0; 2.0; 2.0;;]], [22], 1, :scalarInElements)

In [31]:
I3 = unitTensor(F)
J = det(F)

ScalarField([[0.12; 0.12; … ; 0.12; 0.12;;]], [27], 1, :sc)

In [32]:
sc * I3

LowLevelFEM.TensorField(Matrix{Float64}[], Int64[], 1, :e)

In [33]:
I3 - I3

LowLevelFEM.TensorField([[0.0; 0.0; … ; 0.0; 0.0;;]], [27], 1, :e)

In [34]:
σ = F * SII * F' * inv(detI(F))

reshape(σ.A[1][1:9], 3, 3)

3×3 Matrix{Float64}:
 -162.837     0.0       0.0
    0.0    -236.405     0.0
    0.0       0.0    -326.32

In [35]:
J = 0.5 * 0.5 * 0.5
μ / J * (J^(2 / 3) - 1) + λ / J * log(J)

-119.05114807753088

In [36]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [37]:
gmsh.finalize()